# Import Bibliotheken

In [1]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt

### Package Versionen

In [2]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [3]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [4]:
comment_df = pd.read_excel('data/clean/Google_Rezensionen.xlsx')

In [5]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df[index]['Kommentar']
        return text

### Bestimmung Test- und Traningsdaten

In [6]:
# Länge der Test- und Traningsdaten bestimmen
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [7]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

## Dataloader

### Tokenisierung

In [8]:
# tokenizer = get_tokenizer('spacy', language='de_core_news_lg',)
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_sm')

In [9]:
# Beispiel für Anwendung des tokenizer
sentence = "This isn't a very long example."
tokenizer(sentence)

['This', "isn't", 'a', 'very', 'long', 'example', '.']

### Vokabular erstellen

#### Vokabular wird nicht erstellt !!!

In [10]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [11]:
len(train_set)

3533

In [12]:
tokens = yield_tokens(train_set)

In [15]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [16]:
len(vocab)

1

In [17]:
vocab(['Hallo', 'sehr', 'Personal'])

[0, 0, 0]

In [ ]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [ ]:
onehot = torch.zeros(1,len(vocab))

In [ ]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[0, 0, 0]

In [ ]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Erstellung Dataloader

In [ ]:
label_pipeline = lambda x: int(x) -1

In [ ]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        label_list.append(label_pipeline(_label))
        
        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.tensor(label_list, dtype=torch.int64)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)
    
    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [ ]:
train_loader = DataLoader(
 train_set, batch_size=64,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [ ]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
vocab_size = len(vocab)
num_class = 5
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [ ]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.CrossEntropyLoss()

## Initialisierung Optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate, momentum=0.9)

## Definition der Epochen
num_epochs = 20

In [ ]:
train_accuracy = torchmetrics.Accuracy().to(device)

loss_hist = {}
accuracy_hist = {}

### Training funktioniert nicht !!!

In [ ]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)
        # Der Fehler wird berechnet
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label)
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().item()
    train_accuracy.reset()

KeyError: 392

### Auswertung Fehlerentwicklung

In [ ]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [ ]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
)
# glue('train-loss-team', train_chart,display=True)

### Auswertung Genauigkeit

In [ ]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [ ]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
)